# Retrieval Augmented Generation (RAG)
Retrieval Augmented Generation (RAG) combines information retrieval with language models. It first searches for relevant facts in external sources, then feeds those facts to the language model alongside the user's prompt. This helps the model generate more accurate and factual responses, even on topics beyond its initial training data.


---
## 1.&nbsp; Installations and Settings 🛠️
Let's get started by setting up a GPU on your Colab notebook. Head over to `Edit > Notebook Settings` and select `GPU` from the runtime type dropdown. Once you've made that change, click `Save`.

Now, we'll need to install two additional libraries to build our RAG model. These libraries will help us create and store numerical representations of our text, which are essential for this task.

1. **sentence_transformers:** This library will generate embeddings, which are like numerical summaries of our text. These embeddings will allow us to compare different sentences and identify relationships between them.

2. **faiss-gpu:** This library provides a fast and efficient database for storing and retrieving our numerical summaries.

> If you're using a CPU instead of a GPU, install `faiss-cpu` instead. This version will work just fine, but it may be a bit slower.

In [ ]:
!pip3 install -qqq langchain --progress-bar off
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip3 install -qqq llama-cpp-python --progress-bar off

!pip3 install -qqq sentence_transformers --progress-bar off
!pip3 install -qqq faiss-gpu --progress-bar off

!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF mistral-7b-instruct-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


---
## 2.&nbsp; Setting up your LLM 🧠
Here we'll add one extra parameter, `n_ctx`. This parameter controls the size of the input context. The larger the window, the more memory you're likely to use. The default of 512 is fine for most basic things, but as we are starting to retrieve articles and add them to the context window, we'll double the size to 1024. Feel free to play around with this number as your project needs.

In [ ]:
from langchain.llms import LlamaCpp

llm = LlamaCpp(model_path = "/content/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
               max_tokens = 2000,
               temperature = 0.1,
               top_p = 1,
               n_gpu_layers = -1,
               n_ctx = 1024)


### 2.1.&nbsp;  Test your LLM

In [ ]:
answer_1 = llm.invoke("Write a poem about data science.")
print(answer_1)

---
## 3.&nbsp; Retrieval Augmented Generation 🔃

### 3.1.&nbsp; Find your data
Our model needs some information to work its magic! In this case, we'll be using a copy of Alice's Adventures in Wonderland, but feel free to swap it out for anything you like: legal documents, school textbooks, websites – the possibilities are endless!

In [ ]:
!wget -O /content/alice_in_wonderland.txt https://www.gutenberg.org/cache/epub/11/pg11.txt

> If your working locally, just download a txt book from Project Gutenburg. Here's the link to [Alice's Adventures in Wonderland](https://www.gutenberg.org/cache/epub/11/pg11.txt). Feel free to use any other book though.

### 3.2.&nbsp; Load the data
Now that we have the data, we have to load it in a format LangChain can understand. For this, Langchain has [loaders](https://python.langchain.com/docs/modules/data_connection/document_loaders/). There's loaders for CSV, text, PDF, and a host of other formats. You're not restricted to just text here.



In [ ]:
from langchain.document_loaders import TextLoader

loader = TextLoader("/content/alice_in_wonderland.txt")
documents = loader.load()

### 3.3.&nbsp; Splitting the document
Obviously, a whole book is a lot to digest. This is made easier by [splitting](https://python.langchain.com/docs/modules/data_connection/document_transformers/) the document into chunks. You can split it by paragraphs, sentences, or even individual words, depending on what you want to analyse. In Langchain, we have different tools like the RecursiveCharacterTextSplitter (say that five times fast!) that understand the structure of text and help you break it down into manageable chunks.

Check out [this website](https://chunkviz.up.railway.app/) to help visualise the splitting process.


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=800,
                                               chunk_overlap=150)

docs = text_splitter.split_documents(documents)

### 3.4.&nbsp; Creating vectors with embeddings

[Embeddings](https://python.langchain.com/docs/integrations/text_embedding) are a fancy way of saying we turn words into numbers that computers can understand. Each word gets its own unique code, based on its meaning and relationship to other words. The list of numbers produced is known as a vector. Vectors allow us to compare text and find chunks that contain similar information.

Different embedding models encode words and meanings in different ways, and finding the right one can be tricky. We're using open-source models from HuggingFace, who even have a handy [leaderboard of embeddings](https://huggingface.co/spaces/mteb/leaderboard) on their website. Just browse the options and see which one speaks your language (literally!).
> As we are doing a retrieval project, click on the `Retrieval` tab of the leaderboard to see the best embeddings for retrieval tasks.

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

> The first time you use the embeddings model, it'll download all the necessary data. After that it runs locally on your machine. Unfortunately, Colab is a different story – its sessions end, so the model needs to download again each time.

To exemplify using embeddings to transform a sentence into a vector, let's look at an example:

In [ ]:
test_text = "Why do data scientists make great comedians? They're always trying to make ANOVA pun"
query_result = embeddings.embed_query(test_text)
query_result

In [ ]:
characters = len(test_text)
dimensions = len(query_result)
print(f"The {characters} character sentence was transformed into a {dimensions} dimension vector")

Embedding vectors have a fixed length, meaning each vector produced by this specific embedding will always have 384 dimensions. Choosing the appropriate embedding size involves a trade-off between accuracy and computational efficiency. Larger embeddings capture more semantic information but require more memory and processing power. Start with the provided MiniLM embedding as a baseline and experiment with different sizes to find the optimal balance for your needs.

### 3.5.&nbsp; Creating a vector database
Imagine a library where books aren't just filed alphabetically, but also by their themes, characters, and emotions. That's the magic of vector databases: they unlock information beyond keywords, connecting ideas in unexpected ways.

In [ ]:
from langchain.vectorstores import FAISS

vector_db = FAISS.from_documents(docs, embeddings)

Once the database is made, you can save it to use over and over again in the future.

In [ ]:
vector_db.save_local("/content/faiss_index")

Here's the code to load it again.

We'll leave it commented out here as we don't need it right now - it's already stored above in the variable `vector_db`.

In [ ]:
# new_db = FAISS.load_local("/content/faiss_index", embeddings)

You can also search your database to see which vectors are close to your input.

In [ ]:
vector_db.similarity_search("What does the Mad Hatter drink?")

### 3.6.&nbsp; Adding a prompt
We can guide our model's behavior with a prompt, similar to how we gave instructions to the chatbot. We'll use specific tags in the prompt to tell the model what to do. These tags, `<s> </s>` and `[INST] [/INST]`, come straight from the [model's documentation on Hugging Face](https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF). They can also be found in [Mistral's docs](https://docs.mistral.ai/models/). Different models have different expectations for prompts, so always check the documentation.

In [ ]:
from langchain.prompts.prompt import PromptTemplate

input_template = """
<s>
[INST] Answer the question based only on the following context: [/INST]
{context}
</s>
[INST] Question: {question} [/INST]
"""

prompt = PromptTemplate(template=input_template,
                        input_variables=["context", "question"])

### 3.7.&nbsp; RAG - chaining it all together
This is the final piece of the puzzle, we now bring everything together in a chain. Our vector database, our prompt, and our LLM join to give us retrieval augmented generation.

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_db.as_retriever(search_kwargs={"k": 2}), # top 2 results only, speed things up
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [ ]:
answer = qa_chain.invoke("Who likes to chop off heads?")

answer

#### 3.7.1.&nbsp; Exploring the returned dictionary

In [ ]:
answer.keys()

##### `query`

The question that we asked.

In [ ]:
answer['query']

##### `result`

The response.

In [ ]:
answer['result']

In [ ]:
print(answer['result'])

##### `source_documents`

What information was used to form the response.

In [ ]:
answer['source_documents']

In [ ]:
answer['source_documents'][0]

In [ ]:
answer['source_documents'][0].page_content

In [ ]:
print(answer['source_documents'][0].page_content)

The documents name also gets returned, useful if you have multiple documents!

In [ ]:
answer['source_documents'][0].metadata

In [ ]:
answer['source_documents'][0].metadata["source"]

---
## Challenge 😀
Build a RAG chain that you can use to question the PDF of the python version of `An Introduction to Statistical Learning`. This is an amazing book covering all the maths behind machine learning, you should definitely keep a copy on your hard drive.
* https://www.statlearning.com/
* https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf